In [1]:
import pandas as pd
import cobra
from copy import deepcopy

import sys
sys.path.append('../pycore/')
from utils import extract_details_from_rxnid

In [3]:
prods = ['hdca', 'hxdcol', 'tag', 'citm', 'btoh', 'polyhb', 'arte',
         'sesqt', 'etoh', 'lac__L', 'btd', 'ibutoh', '3hppa', '3hppb',
         'succ', 'mal__L', 'skm', 'muco', '4hbz', '4abz', '2phetoh',
         'styr', 'cou', 'retcln', 'nrgn', 'rsvtol', 'glyc', '13ppd']
prod_rxns = {p:[] for p in prods}

#### Compile product - active rxns dictionary
df_rxns = pd.read_excel('./input/pathways.xlsx')
for i in df_rxns.index:
    rxnid = extract_details_from_rxnid(df_rxns.id[i])[1]
    p = df_rxns.loc[i, 'product']
    
    if p == '3hpp':
        prod_rxns['3hppa'].append(rxnid)
        prod_rxns['3hppb'].append(rxnid)
    else:
        prod_rxns[p].append(rxnid)

In [4]:
df_comp = pd.read_excel('./compiled_results.xlsx')
df_comp.index = df_comp['product'].to_list()

In [5]:
for p in prods:
    print(df_comp.loc[p, 'MW (g/mol)'])

256.4
242.44
824.3
146.1
74.12
103.1
234.33
246.3
46.07
89.07
90.12
74.12
90.08
90.08
116.07
132.07
174.15
140.09
137.11
137.14
122.16
104.15
163.15
329.4
272.25
228.24
92.09
76.09


In [7]:
model = cobra.io.load_json_model('./input/GSM_iSace1144_rba_pathwayadd.json')
model.solver = 'cplex'
model.objective = dict()

model.reactions.ATPM_c.lower_bound = 1

model.reactions.EX_glc__D_e.bounds = (-16,1000)
model.reactions.BIOMASS_AERO_SC_hvd.bounds = (0.1, 0.1)
model.reactions.EX_o2_e.bounds = (-1000,1000)
model.reactions.EX_co2_e.bounds = (-1000,1000)

# Allow YNB media uptake
model.reactions.EX_thm_e.bounds = (-1000,1000)
model.reactions.EX_ribflv_e.bounds = (-1000,1000)
model.reactions.EX_nac_e.bounds = (-1000,1000)
model.reactions.EX_pydxn_e.bounds = (-1000,1000)
model.reactions.EX_fol_e.bounds = (-1000,1000)
model.reactions.EX_pnto__R_e.bounds = (-1000,1000)
model.reactions.EX_4abz_e.bounds = (-1000,1000)
model.reactions.EX_inost_e.bounds = (-1000,1000)


--------------------------------------------
--------------------------------------------

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2022-08-08


In [8]:
fba_yields = {p:0 for p in prods}

for p in prods:
    if p in ['3hppa', '3hppb']:
        exrxn = 'EX_3hpp_e'
    else:
        exrxn = 'EX_' + p + '_e'
    if p in ['citm', 'polyhb']:
        model.reactions.THRA_c.knock_out()
    for r in prod_rxns[p]:
        model.reactions.get_by_id(r).bounds = (0,1000)
    model.reactions.get_by_id(exrxn).objective_coefficient = 1
    fba = model.optimize()
    
    for r in prod_rxns[p]:
        model.reactions.get_by_id(r).bounds = (0,0)
    if p in ['citm', 'polyhb']:
        model.reactions.THRA_c.bounds = (0,1000)
    model.objective = dict()
    
    #fba_yields[p] = fba[exrxn]
    #print(p, fba.status, fba[exrxn], fba['EX_glc__D_e'])
    print(fba[exrxn])

3.38795275079586
3.2839541127181757
1.0423506529501554
15.279370538802125
13.735847934428225
15.279370538801352
3.485763579142557
3.623945922817035
30.627707005199625
29.757146187690683
15.74955936955675
15.31385350259982
24.626603741537078
27.404340168199415
23.544115669049752
26.128225925408767
11.936014071774578
12.791131497812355
11.108317096749033
10.598754707586854
8.518944215959147
8.30301233618416
8.705819119942682
3.077937328691979
4.864933942163164
4.864933942688053
24.503199547020625
21.862393131268842


In [14]:
p = 'citm'
if p in ['3hppa', '3hppb']:
    exrxn = 'EX_3hpp_e'
else:
    exrxn = 'EX_' + p + '_e'

model.reactions.EX_glc__D_e.bounds = (-16, 1000)
for r in prod_rxns[p]:
    model.reactions.get_by_id(r).bounds = (0,1000)
model.reactions.get_by_id(exrxn).objective_coefficient = 1
fba = model.optimize()

for r in prod_rxns[p]:
    model.reactions.get_by_id(r).bounds = (0,0)
model.objective = dict()

#fba_yields[p] = fba[exrxn]
#print(p, fba.status, fba[exrxn], fba['EX_glc__D_e'])
print(fba[exrxn], fba[exrxn] * 146.1 / 180.156 / fba['EX_glc__D_e'])

15.905128256361175 -0.806155234301927


In [5]:
fba['EX_thm_e']

-1e-07

In [21]:
for p in prods:
    print(p)

hdca
hxdcol
btoh
tag
nrgn
polyhb
citm
arte
sesqt
etoh
lac__L
btd
ibutoh
3hppa
3hppb
succ
mal__L
skm
muco
4hbz
4abz
2phetoh
cou
rsvtol
styr
retcln
glyc
13ppd
